In [3]:
# !pip install pillow numpy pycryptodome

In [17]:
from PIL import Image
import numpy as np
from Crypto.Cipher import AES
import os

# Configuración
input_image_path = 'tux.bmp'
output_image_ecb_path = 'tux_encrypted_ecb.png'
output_image_cbc_path = 'tux_encrypted_cbc.png'
image_shape = (405, 480, 4) # Dimensiones reshape RGBA
key = os.urandom(16)        # Clave de 128 bits (16 bytes)
iv = os.urandom(16)         # Vector de inicialización para CBC de 16 bytes

# 1. Cargar la imagen y convertirla a bytes
img = Image.open(input_image_path).convert('RGBA')
img_array = np.array(img)

# Redimensionar si es necesario
if img_array.shape != image_shape:
    img_array = np.resize(img_array, image_shape)

# Aplanar la imagen a bytes
img_bytes = img_array.tobytes()

# Agregar padding
padding_length = 16 - (len(img_bytes) % 16)
img_bytes_padded = img_bytes + bytes([padding_length] * padding_length)

# 2. Cifrado en Modo ECB
cipher_ecb = AES.new(key, AES.MODE_ECB)
encrypted_bytes_ecb = cipher_ecb.encrypt(img_bytes_padded)

# Convertir los bytes cifrados ECB a una imagen y guardarla
encrypted_array_ecb = np.frombuffer(encrypted_bytes_ecb[:np.prod(image_shape)], dtype=np.uint8)
encrypted_array_ecb = encrypted_array_ecb.reshape(image_shape)
encrypted_img_ecb = Image.fromarray(encrypted_array_ecb, 'RGBA')
encrypted_img_ecb.save(output_image_ecb_path)
print(f"Imagen cifrada en modo ECB > {output_image_ecb_path}")

# 3. Cifrado en Modo CBC
cipher_cbc = AES.new(key, AES.MODE_CBC, iv)
encrypted_bytes_cbc = cipher_cbc.encrypt(img_bytes_padded)

# Convertir los bytes cifrados CBC a una imagen y guardarla
encrypted_array_cbc = np.frombuffer(encrypted_bytes_cbc[:np.prod(image_shape)], dtype=np.uint8)
encrypted_array_cbc = encrypted_array_cbc.reshape(image_shape)
encrypted_img_cbc = Image.fromarray(encrypted_array_cbc, 'RGBA')
encrypted_img_cbc.save(output_image_cbc_path)
print(f"Imagen cifrada en modo CBC > {output_image_cbc_path}")


Imagen cifrada en modo ECB > tux_encrypted_ecb.png
Imagen cifrada en modo CBC > tux_encrypted_cbc.png
